In [1]:
from selenium import webdriver
from bs4  import BeautifulSoup
import pandas as pd

import requests
import os

In [109]:
import re
driver = webdriver.Chrome(r"C:\Users\User\Downloads\Compressed\chromedriver_win32_2\chromedriver.exe")

srch_url="https://www.flipkart.com/womens-dresses/pr?sid=clo%2Codx%2Cmaj%2Cjhy&marketplace=FLIPKART&otracker=product_breadCrumbs_Women%27s+Dresses"


In [168]:
def down_img(url,scd):
    img_data = requests.get(url).content
    if not os.path.exists('downloaded_image'):
        os.makedirs('downloaded_image')
    else:
        img_name=os.getcwd()+'/downloaded_image'+'/image_SC_%s_.jpg'%re.sub('\W+',' ',scd )
        with open(img_name, 'wb') as handler:
            handler.write(img_data)
#             print(img_name+'  Image downloaded')
        return './downloaded_image'+'/image_SC%s_.jpg'%scd

In [173]:
class scrape_one_prod:
#     global otdic
    def __init__(self):
        self.name= list()
        self.price = list()
        self.dis = list()
        self.url = []
        self.df_p= pd.DataFrame()
        
    def scrape_details(self,srch_url,pg_no):
        driver.get(srch_url+r'&page='+str(pg_no))
        content = driver.page_source
        soup = BeautifulSoup(content)
        url = list()
#         oth_dic ={}
        for i,a in enumerate(soup.findAll(attrs={'class':'_2B099V'})):
#             print(a.find('div',attrs={'class':'_2pi5LC'}))
            self.url=a.find('a',attrs='IRpwTa')['href']
            url.append('https://www.flipkart.com'+self.url)
            
            self.scrape_product(str(url[i]),a)
#         return oth_dic
    
    def scrape_product(self,url_,a):
#         print(url_)
        driver.get(url_)
        content_=driver.page_source
        soup = BeautifulSoup(content_)
        otdic = dict()
        dec,colr_,col,siz = str(),str(),list(),list()
        #Description
        self.name=a.find('a',attrs='IRpwTa')
#         print(self.name.text)
        self.dis=a.find('div',attrs='_3Ay6Sb')
        self.price=a.find('div',attrs='_30jeq3')
        rat = str()
        if a.find('div',attrs='_3LWZlK _3uSWvT'):rat = a.find('div',attrs='_3LWZlK _3uSWvT').text;
#         self.url='https://www.flipkart.com'+a.find('a',attrs='IRpwTa')['href']
        
        otdic.update({'Product_name':self.name,'Price':self.price,'Didcount':self.dis,'P_URL':url_,'Rating':rat})
        
        #Description
        if soup.find('div',attrs='_1AN87F'):
            dec = soup.find('div',attrs='_1AN87F').text
#             otdic.update({'Description':dec})
#             print('Product Decription found :'+otdic['Description']
        else:
            dec = None
#             otdic.update({'Description':'None'})
            
        #Color    
        if soup.find('li',attrs={'id':re.compile('-color')}):
            for itm in soup.find_all('li','_3V2wfe _31hAvz'):
                if itm.find('a','kmlXmn PP89tw'):
            #         print(itm.find('a','kmlXmn PP89tw')['href'])
            #         print(itm.find('div','_3Oikkn _3_ezix _2KarXJ _31hAvz').text)
                    colr_ = itm.find('div','_3Oikkn _3_ezix _2KarXJ _31hAvz').text
#                     otdic.update({'P_Color':colr_})
#                     print('Product Color :'+otdic['P_Color'])
        else:
            colr_=None
#             otdic.update({'P_Color':'None'})
            
         
        #Available colors
#         col=[]
        if soup.find('li',{'id':re.compile('-color')}):
            for items in soup.find_all('li',{'class':'_3V2wfe _31hAvz','id':re.compile('-color')}):
                col.append(items.find('div','_3Oikkn _3_ezix _2KarXJ _31hAvz').text)
            #     print(items.find('div','_3Oikkn _3_ezix _2KarXJ _31hAvz').text)
#             otdic.update({'Avl_Color':col})
#             print('Available COlors ',end='\t')
#             print(otdic['Avl_Color'])
        else:
            col =[None]
#             otdic.update({'Avl_Color':'None'})
        
        #Available sizes
#         siz = []
        if soup.find('li',{'id':re.compile('-size')}):
            for items in soup.find_all('li',{'class':'_3V2wfe _31hAvz','id':re.compile('-size')}):
                siz.append(items.find('div',attrs={'class':re.compile('_3Oikkn _3_ezix _2KarXJ _31hAvz')}).text)
            #     print(items.find('div','_3Oikkn _3_ezix _2KarXJ _31hAvz').text)            
#             otdic.update({'Avl_Size':siz})
#             print('Available Size ',end='\t')
#             print(otdic['Avl_Size']) 
        else:
            siz = [None]
#             otdic.update({'Avl_Size':'None'})
        
        #Image URL
#         if soup.find('img','_2r_T1I _396QI4'):
        img_url_ = soup.find('img','_2r_T1I _396QI4')['src']
        
        ot_ky, ky_vl = [],[]
        for k,v in zip(soup.find_all('div',attrs=['_2H87wv',]), soup.find_all('div',attrs=['_2vZqPX'])):
#             print(k.text+': '+v.text)
#             otdic.update({k.text:v.text})
            ot_ky.append(k.text)
            ky_vl.append(v.text)
        otdic.update({k__:v__ for k__, v__ in zip(ot_ky,ky_vl)})
#         print(otdic)
        im_path = down_img(img_url_,otdic['Style Code'])
        otdic.update({'Description': dec, 'P_Color':colr_ , 'Avl_Color': [col], 'Avl_Size':[siz] ,'Image_path':im_path})
        self.df_p = pd.DataFrame.from_dict(otdic)
        global df
        df = pd.concat([df,self.df_p])
#         return otdic
#         pass
        

In [174]:
global df
df = pd.DataFrame()
scrapper = scrape_one_prod()

In [176]:

for i in range(1,4):
    ot_dic = scrapper.scrape_details(srch_url,i)
#     print(srch_url+str(i))
#     dic = {'Product Name':name,'Price':price,'Rating':dis,'url':url}
#     dic.update
    df = df.append(pd.DataFrame(ot_dic),ignore_index=True)
    print('Extracted %d Page products and total %d products '%(i,df.shape[0]))

Extracted 1 Page products and total 40 products 
Extracted 2 Page products and total 80 products 
Extracted 3 Page products and total 120 products 


# Analyse products

In [177]:
df.columns

Index(['Product_name', 'Price', 'Didcount', 'P_URL', 'Rating', 'Color',
       'Length', 'Fabric', 'Pattern', 'Ideal For', 'Type', 'Style Code',
       'Suitable For', 'Sleeve Length', 'Pack of', 'Neck', 'Fabric Care',
       'Other Details', 'Sleeve', 'Generic Name', 'Country of Origin',
       'Description', 'P_Color', 'Avl_Color', 'Avl_Size', 'Image_path',
       'Lining Material'],
      dtype='object')

In [179]:
df['Color'].value_counts()

Black                                 20
Multicolor                            18
Pink                                  15
Green                                 12
Blue                                  12
Yellow                                 8
Maroon                                 5
Red                                    4
Dark Green                             4
Purple                                 3
White, Blue                            3
Dark Blue                              2
White, Black                           2
White                                  2
Light Blue                             2
Orange                                 2
Maroon, Gold                           1
Dark Blue, Red, Gold                   1
Beige, Black                           1
Grey                                   1
Multicolor, Dark Blue, Pink, Beige     1
Maroon, White                          1
Name: Color, dtype: int64

In [180]:
df.head()

,Product_name,Price,Didcount,P_URL,Rating,Color,Length,Fabric,Pattern,Ideal For,...,Other Details,Sleeve,Generic Name,Country of Origin,Description,P_Color,Avl_Color,Avl_Size,Image_path,Lining Material
0,Women A-line Green Dress,₹529,"(47% off,)",https://www.flipkart.com/iqra-fashion-women-a-...,,Green,Maxi/Full Length,Poly Crepe,Floral Print,Women,...,Nice Addition To The Wardrobe Of Every Fashion...,Half Sleeve,Dress,India,"Made of Premium fabric, these dresses provided...",Green,"[Black, Dark Green, Green, Pink]","[Waist 33 inch | Bust 36 inch, Waist 35 inch |...",./downloaded_image/image_SCFID-21-Green_FLO_.jpg,NaN
1,Women A-line Pink Dress,₹529,"(47% off,)",https://www.flipkart.com/iqra-fashion-women-a-...,,Pink,Maxi/Full Length,Poly Crepe,Floral Print,Women,...,Nice Addition To The Wardrobe Of Every Fashion...,Half Sleeve,Dress,India,"Made of Premium fabric, these dresses provided...",Pink,"[Black, Dark Green, Green, Pink]","[Waist 33 inch | Bust 36 inch, Waist 35 inch |...",./downloaded_image/image_SCFID-21-Rose_.jpg,NaN
2,Women Fit and Flare Blue Dress,₹225,"(74% off,)",https://www.flipkart.com/digimart-women-fit-fl...,,Blue,Below Knee,Cotton Blend,NaN,Women,...,NaN,NaN,Dress,India,DIGIMART WOMEN FANCY DRESS,None,[None],"[S, Bust 38 inch, Bust 40 inch, Bust 42 inch]",./downloaded_image/image_SCNWBFR001_.jpg,NaN
3,Women Fit and Flare Multicolor Dress,₹599,"(45% off,)",https://www.flipkart.com/aayu-women-fit-flare-...,,Multicolor,Midi/Calf Length,Poly Crepe,Printed,Women,...,NaN,Half Sleeve,NaN,NaN,BOHO LONG DRESS,None,[None],"[Waist 27 inch | Bust 35 inch, Waist 29 inch |...",./downloaded_image/image_SCAA#0119 BOHO LONG D...,NaN
4,"Women Gown Dark Blue, Red, Gold Dress",₹379,"(81% off,)",https://www.flipkart.com/wear-shine-women-gown...,,"Dark Blue, Red, Gold",Maxi/Full Length,Satin Blend,Embroidered,Women,...,NaN,Regular Sleeves,Dress,India,Experience the exotic floral bliss in soft des...,None,[None],[Bust 42 inch],./downloaded_image/image_SCMR21_.jpg,NaN


In [181]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120 entries, 0 to 119
Data columns (total 27 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Product_name       120 non-null    object
 1   Price              120 non-null    object
 2   Didcount           120 non-null    object
 3   P_URL              120 non-null    object
 4   Rating             120 non-null    object
 5   Color              120 non-null    object
 6   Length             120 non-null    object
 7   Fabric             120 non-null    object
 8   Pattern            105 non-null    object
 9   Ideal For          120 non-null    object
 10  Type               120 non-null    object
 11  Style Code         120 non-null    object
 12  Suitable For       120 non-null    object
 13  Sleeve Length      82 non-null     object
 14  Pack of            111 non-null    object
 15  Neck               94 non-null     object
 16  Fabric Care        107 non-null    object
 1